<a href="https://colab.research.google.com/github/OleSpooky/README/blob/main/2025.11.29_RAW_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Persistence Simulation - Interactive Notebook

## Overview
This notebook implements a vectorized Ising-like model for analyzing information persistence across coupled node networks. It explores how "information pockets" with strong coupling retain information longer than surrounding regions.

## Environment Requirements

### Python Dependencies
All dependencies are managed in `requirements.txt`. Install via:
```bash
pip install -r requirements.txt
```

**Required versions:**
- `numpy >= 1.20.0` - Uses `np.random.default_rng` API (introduced in NumPy 1.17)
- `scipy >= 1.7.0` - For `scipy.special.expit` (sigmoid function)
- `pandas >= 1.3.0` - For data analysis and CSV handling
- `matplotlib >= 3.4.0` - For visualization
- `ipywidgets >= 7.6.0` - For interactive sliders (compatible with @jupyter-widgets/controls v1.5.0)

### Supported Environments
- ✅ **Google Colab** (recommended for interactive widgets)
- ✅ **Jupyter Notebook**
- ✅ **JupyterLab**
- ✅ **Local Python** (non-interactive mode)

## Execution Order

**For a complete workflow, run cells in this order:**

1. **Cell 1**: Import dependencies and define simulation functions
2. **Cell 3**: *(Optional)* Verify dependencies installed
3. **Cell 4**: Run experiment runner to generate data (creates `colab_results/` folder)
4. **Cell 5**: Combine CSV results and display summary
5. **Cell 6**: Analyze persistence times
6. **Cell 7**: Visualize total information flow
7. **Cell 8-9**: *(Optional)* Mount Google Drive (Colab only)
8. **Cell 13-14**: Interactive widget explorer (requires previous cells)

## Key Features

### 1. Vectorized Batch Simulation
- Efficient parallel computation of M ensemble members
- Memory-efficient chunking for large ensembles
- Reproducible results with seed control

### 2. Information Metrics
- **Mutual Information I(A:X_i,t)**: Measures correlation between source and each node
- **Persistence Time (τ)**: Time until MI drops below threshold
- **Pocket Analysis**: Compare information retention inside vs. outside pockets

### 3. Interactive Exploration
- Real-time parameter tuning with sliders
- Visual feedback: heatmaps, time series, and node-wise persistence
- Parameter ranges: pocket strength (0-40), beta/inverse temperature (0.2-4.0)

## Cross-Platform Compatibility

- **File paths**: Uses `os.path.join()` for OS-agnostic path handling
- **Folder creation**: Idempotent with `os.makedirs(folder, exist_ok=True)`
- **Error handling**: CSV/NPY loading includes try-except blocks for robustness

## Output Data

Generated files are saved to `colab_results/`:
- `tau_{config_name}.csv` - Persistence times for each node
- `I_{config_name}.npy` - Mutual information matrix (T × N)

---

**Ready to begin? Run Cell 1 to load simulation functions.**


In [ ]:
# ==========================================
# DEPENDENCIES AND IMPORTS
# ==========================================
# All required libraries are specified in requirements.txt
# Install via: pip install -r requirements.txt
# Core dependencies: numpy>=1.20.0, scipy>=1.7.0, pandas>=1.3.0

import numpy as np
import pandas as pd
import os
import glob
from scipy.special import expit  # Sigmoid function for probability calculations

# ==========================================
# PART 1: VECTORIZED SIMULATOR (The Engine)
# ==========================================
# This section implements a vectorized Ising-like model for information
# persistence analysis across a network of nodes. The simulation tracks
# how information propagates and decays through coupled nodes over time.

def init_equilibrium_batch(N, M, rng):
    """
    Initialize equilibrium states for a batch of simulations.

    Args:
        N (int): Number of nodes in the network
        M (int): Ensemble size (number of parallel simulations)
        rng: Numpy random generator

    Returns:
        np.ndarray: Initial states (M x N) with random binary values
    """
    return rng.integers(0, 2, size=(M, N), dtype=np.int8)

def update_step_batch(X, c_arr, beta, theta_arr, rng):
    """
    Update network states for one timestep using Ising-like dynamics.

    This function computes the probability of each node being in state 1
    based on its neighbors' states, coupling strengths, and temperature.

    Args:
        X (np.ndarray): Current states (M x N)
        c_arr (np.ndarray): Coupling strengths between adjacent nodes (length N-1)
        beta (float): Inverse temperature (controls noise/randomness)
        theta_arr (np.ndarray): Bias/threshold for each node (length N)
        rng: Numpy random generator

    Returns:
        np.ndarray: Updated states (M x N)
    """
    M, N = X.shape

    # Calculate neighbor contributions (left and right neighbors)
    left = np.zeros_like(X)
    right = np.zeros_like(X)
    left[:, 1:] = X[:, :-1] * c_arr[:N-1]   # Left neighbor influence
    right[:, :-1] = X[:, 1:] * c_arr[:N-1]  # Right neighbor influence

    # Compute total neighbor influence
    neighbor_sum = left + right

    # Calculate transition probability using sigmoid (expit)
    bias = beta * (neighbor_sum - theta_arr[None, :])
    p1 = expit(bias)  # Probability of being in state 1

    # Stochastic update based on computed probabilities
    U = rng.random(size=(M, N))
    return (U < p1).astype(np.int8)

def run_simulation_batch(N, c_arr, beta, theta_arr, M, T, source_j, master_seed=12345):
    """
    Run batch simulation and collect statistics for mutual information calculation.

    This is the main simulation driver that tracks co-occurrence counts between
    the source node and all other nodes over time, which are used to compute
    mutual information I(A:X_i,t).

    Args:
        N (int): Number of nodes
        c_arr (np.ndarray): Coupling strengths (length N-1)
        beta (float): Inverse temperature
        theta_arr (np.ndarray): Node biases (length N)
        M (int): Ensemble size (number of runs)
        T (int): Number of timesteps
        source_j (int): Index of the source node
        master_seed (int): Random seed for reproducibility

    Returns:
        np.ndarray: Co-occurrence counts (T x N x 2 x 2)
                   Format: counts[t, node, source_state, node_state]
    """
    rng = np.random.default_rng(master_seed)

    # Initialize count array for mutual information calculation
    counts = np.zeros((T, N, 2, 2), dtype=np.int64)

    # Initialize equilibrium states
    X = init_equilibrium_batch(N, M, rng)

    # Set source node to random binary value
    A = rng.integers(0, 2, size=M, dtype=np.int8)
    X[:, source_j] = A

    # Run simulation and collect statistics
    for t in range(T):
        # Collect co-occurrence counts for this timestep
        for a in (0, 1):
            mask_a = (A == a)
            if not np.any(mask_a):
                continue
            sub = X[mask_a]
            ones = np.sum(sub, axis=0)
            zeros = sub.shape[0] - ones
            counts[t, :, a, 1] += ones
            counts[t, :, a, 0] += zeros

        # Update states for next timestep
        X = update_step_batch(X, c_arr, beta, theta_arr, rng)

    return counts

def compute_mi_from_counts(counts, M):
    """
    Compute mutual information I(A:X_i,t) from co-occurrence counts.

    Mutual information quantifies how much knowing the source state tells
    us about each node's state at each time. Higher MI indicates stronger
    correlation and information retention.

    Args:
        counts (np.ndarray): Co-occurrence counts (T x N x 2 x 2)
        M (int): Ensemble size (for normalization)

    Returns:
        np.ndarray: Mutual information matrix (T x N)
                   I[t, i] = I(A : X_i, t) in bits
    """
    T, N, _, _ = counts.shape
    I = np.zeros((T, N), dtype=float)
    epsilon = 1e-12  # Small constant to avoid log(0)

    for t in range(T):
        for i in range(N):
            # Compute joint probability p(a, x)
            p_ax = counts[t, i].astype(float) / M

            # Compute marginal probabilities
            p_a = p_ax.sum(axis=1)  # p(source state)
            p_x = p_ax.sum(axis=0)  # p(node state)

            # Calculate mutual information using I(A:X) = Σ p(a,x) log[p(a,x)/(p(a)p(x))]
            mi = 0.0
            for a_val in (0, 1):
                for x_val in (0, 1):
                    p = p_ax[a_val, x_val]
                    if p > epsilon:
                        denom = p_a[a_val] * p_x[x_val]
                        if denom > epsilon:
                            mi += p * np.log2(p / denom)
            I[t, i] = mi

    return I

def compute_tau(I_matrix, threshold=1e-2):
    """
    Compute persistence time (tau) for each node.

    Tau is defined as the first timestep where mutual information drops
    below the threshold, indicating information loss/decay.

    Args:
        I_matrix (np.ndarray): Mutual information over time (T x N)
        threshold (float): MI threshold for considering information lost

    Returns:
        np.ndarray: Persistence times for each node (length N)
                   tau[i] = first time where I[t,i] < threshold
    """
    T, N = I_matrix.shape
    taus = np.zeros(N, dtype=int)

    for i in range(N):
        # Find first timestep where MI drops below threshold
        decayed = np.where(I_matrix[:, i] < threshold)[0]
        if len(decayed) > 0:
            taus[i] = decayed[0]
        else:
            taus[i] = T  # Information never decayed within simulation time

    return taus


In [ ]:
# Dependencies are managed in requirements.txt
# Install via: pip install -r requirements.txt
# Required: numpy>=1.20.0, scipy>=1.7.0, pandas>=1.3.0, matplotlib>=3.4.0

In [ ]:
# ==========================================
# PART 2: EXPERIMENT RUNNER (Data Generation)
# ==========================================

# Settings
N = 21
M = 2000        # Number of runs per batch
T = 60          # Timesteps
source_j = 10   # Source in the middle
beta = 2.0
theta_arr = np.ones(N) * 1.0
folder = 'colab_results'

# Ensure clean start
# Create folder if it doesn't exist (idempotent)
os.makedirs(folder, exist_ok=True)

# Define 3 scenarios to demonstrate the contrast
scenarios = [
    ("homogenous", np.ones(N-1) * 1.0),             # Baseline
    ("p10.0_weak",  np.ones(N-1) * 1.0),            # Will modify below
    ("p10.0_strong", np.ones(N-1) * 1.0)            # Will modify below
]

# Apply modifications for pockets
# Pocket at index 10 means couplings (9-10) and (10-11) are strong
c_weak = np.ones(N-1) * 1.0
c_weak[9:11] = 3.0  # Weak pocket
scenarios[1] = ("beta2.0_p10.0_weak", c_weak)

c_strong = np.ones(N-1) * 1.0
c_strong[9:11] = 5.0 # Strong pocket
scenarios[2] = ("beta2.0_p10.0_strong", c_strong)

print(f"Running {len(scenarios)} simulations...")

for name, c_profile in scenarios:
    print(f"  -> Simulating: {name}")
    counts = run_simulation_batch(N, c_profile, beta, theta_arr, M, T, source_j)
    I = compute_mi_from_counts(counts, M)
    taus = compute_tau(I, threshold=0.01)

    # Save the I array for further analysis
    np.save(os.path.join(folder, f'I_{name}.npy'), I)

    # Save to CSV in the format expected by your analysis script
    # Structure: node_index, tau
    df = pd.DataFrame({
        'node_index': np.arange(N),
        'tau': taus
    })
    filename = os.path.join(folder, f'tau_{name}.csv')
    df.to_csv(filename, index=False)

print("Data generation complete.\n")

In [ ]:
# Paste and run: combine all tau CSVs into one DataFrame and show summary
import os, glob, pandas as pd, numpy as np

folder = 'colab_results'

# Create folder if it doesn't exist
os.makedirs(folder, exist_ok=True)

files = sorted(glob.glob(os.path.join(folder, 'tau_*.csv')))
dfs = []

# Process each CSV with error handling for malformed files
for f in files:
    try:
        cfg = os.path.basename(f).replace('tau_','').replace('.csv','')
        df = pd.read_csv(f)
        df['config'] = cfg
        dfs.append(df)
    except Exception as e:
        print(f"Warning: Could not read {f}: {e}")
        continue

if dfs:
    all_tau = pd.concat(dfs, ignore_index=True)
    display(all_tau.head(40))
    # compute mean tau inside/outside pocket if pockets exist
    print(f"\nSummary: {len(dfs)} CSV files combined, {len(all_tau)} total samples")
else:
    print("No CSV files found in", folder)


In [ ]:
# ==========================================
# PART 3: YOUR ANALYSIS SCRIPT (Data Processing)
# ==========================================
# This cell analyzes persistence times from generated CSV files
# and visualizes information retention across different network configurations

print("--- RUNNING USER ANALYSIS ---")

# Read all tau CSV files with error handling
files = sorted(glob.glob(os.path.join(folder, 'tau_*.csv')))
dfs = []

for f in files:
    try:
        # Extract config name from filename (e.g., 'tau_homogenous.csv' -> 'homogenous')
        cfg = os.path.basename(f).replace('tau_', '').replace('.csv', '')
        df = pd.read_csv(f)
        df['config'] = cfg
        dfs.append(df)
        print(f"✓ Loaded {cfg}: {len(df)} samples")
    except Exception as e:
        print(f"⚠ Warning: Could not read {os.path.basename(f)}: {e}")
        continue

if dfs:
    # Combine all dataframes
    all_df = pd.concat(dfs, ignore_index=True)

    # Display summary statistics
    print(f"\n{'='*60}")
    print(f"Summary: {len(dfs)} configurations, {len(all_df)} total samples")
    print(f"{'='*60}")

    # Group by configuration and compute statistics
    summary = all_df.groupby('config')['tau'].agg(['mean', 'std', 'min', 'max', 'count'])
    print(summary)
else:
    print(f"⚠ No valid CSV files found in '{folder}'. Run data generation first.")


In [ ]:
# ==========================================
# ANALYSIS: Visualize Total Information Flow
# ==========================================
# Compute and plot Σ_i I(A:i,t) - the sum of mutual information
# across all nodes over time. This shows total information retention.

import os, glob, numpy as np, matplotlib.pyplot as plt

folder = 'colab_results'
npy_files = sorted(glob.glob(os.path.join(folder, 'I_*.npy')))

if npy_files:
    plt.figure(figsize=(8, 5))

    for f in npy_files:
        try:
            # Load mutual information matrix (shape: T x N)
            I = np.load(f)

            # Sum across all nodes to get total information at each timestep
            s = I.sum(axis=1)

            # Extract label from filename for plot legend
            label = os.path.basename(f).replace('I_', '').replace('.npy', '')
            plt.plot(s, label=label, linewidth=2)

        except Exception as e:
            print(f"⚠ Warning: Could not load {os.path.basename(f)}: {e}")
            continue

    plt.xlabel('Timestep', fontsize=12)
    plt.ylabel('Sum I(A:i,t)', fontsize=12)
    plt.title('Total Information Retention Over Time', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠ No NPY files found in '{folder}'. Run data generation first.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Google Drive Integration (Colab Only)

After running the above cell and following the authentication steps, your Google Drive will be mounted at `/content/drive`.

**Access patterns:**
```python
# Example: Save to Drive
import pandas as pd
df.to_csv('/content/drive/My Drive/colab_results/data.csv')

# Example: Load from Drive
data = pd.read_csv('/content/drive/My Drive/your_folder/your_file.csv')
```

**Note:** This cell only works in Google Colab. For local Jupyter, save files to the local filesystem instead.


In [ ]:
# Widget dependencies are managed in requirements.txt
# Install via: pip install -r requirements.txt
# Required: ipywidgets>=7.6.0 (@jupyter-widgets/controls v1.5.0 compatible)

In [ ]:
# ==========================================
# PART 2: EXPERIMENT RUNNER (Data Generation)
# ==========================================

# Settings
N = 21
M = 2000        # Number of runs per batch
T = 60          # Timesteps
source_j = 10   # Source in the middle
beta = 2.0
theta_arr = np.ones(N) * 1.0
folder = 'colab_results'

# Ensure clean start
# Create folder if it doesn't exist (idempotent)
os.makedirs(folder, exist_ok=True)

# Define 3 scenarios to demonstrate the contrast
scenarios = [
    ("homogenous", np.ones(N-1) * 1.0),             # Baseline
    ("p10.0_weak",  np.ones(N-1) * 1.0),            # Will modify below
    ("p10.0_strong", np.ones(N-1) * 1.0)            # Will modify below
]

# Apply modifications for pockets
# Pocket at index 10 means couplings (9-10) and (10-11) are strong
c_weak = np.ones(N-1) * 1.0
c_weak[9:11] = 3.0  # Weak pocket
scenarios[1] = ("beta2.0_p10.0_weak", c_weak)

c_strong = np.ones(N-1) * 1.0
c_strong[9:11] = 5.0 # Strong pocket
scenarios[2] = ("beta2.0_p10.0_strong", c_strong)

print(f"Running {len(scenarios)} simulations...")

for name, c_profile in scenarios:
    print(f"  -> Simulating: {name}")
    counts = run_simulation_batch(N, c_profile, beta, theta_arr, M, T, source_j)
    I = compute_mi_from_counts(counts, M)
    taus = compute_tau(I, threshold=0.01)

    # Save the I array for further analysis
    np.save(os.path.join(folder, f'I_{name}.npy'), I)

    # Save to CSV in the format expected by your analysis script
    # Structure: node_index, tau
    df = pd.DataFrame({
        'node_index': np.arange(N),
        'tau': taus
    })
    filename = os.path.join(folder, f'tau_{name}.csv')
    df.to_csv(filename, index=False)

print("Data generation complete.\n")

In [ ]:
# ==========================================
# PART 3: YOUR ANALYSIS SCRIPT (Data Processing)
# ==========================================
# This cell analyzes persistence times from generated CSV files
# and visualizes information retention across different network configurations

print("--- RUNNING USER ANALYSIS ---")

# Read all tau CSV files with error handling
files = sorted(glob.glob(os.path.join(folder, 'tau_*.csv')))
dfs = []

for f in files:
    try:
        # Extract config name from filename (e.g., 'tau_homogenous.csv' -> 'homogenous')
        cfg = os.path.basename(f).replace('tau_', '').replace('.csv', '')
        df = pd.read_csv(f)
        df['config'] = cfg
        dfs.append(df)
        print(f"✓ Loaded {cfg}: {len(df)} samples")
    except Exception as e:
        print(f"⚠ Warning: Could not read {os.path.basename(f)}: {e}")
        continue

if dfs:
    # Combine all dataframes
    all_df = pd.concat(dfs, ignore_index=True)

    # Display summary statistics
    print(f"\n{'='*60}")
    print(f"Summary: {len(dfs)} configurations, {len(all_df)} total samples")
    print(f"{'='*60}")

    # Group by configuration and compute statistics
    summary = all_df.groupby('config')['tau'].agg(['mean', 'std', 'min', 'max', 'count'])
    print(summary)
else:
    print(f"⚠ No valid CSV files found in '{folder}'. Run data generation first.")


In [ ]:
# ==========================================
# INTERACTIVE WIDGET EXPLORER
# ==========================================
# This cell provides an interactive interface for exploring parameter space
# using ipywidgets. Requires: ipywidgets>=7.6.0 (@jupyter-widgets/controls v1.5.0)
#
# ENVIRONMENT NOTES:
# - Works in Jupyter Notebook, JupyterLab, and Google Colab
# - Widget version compatibility: @jupyter-widgets/controls 1.5.0
# - NumPy version: >=1.20.0 (for np.random.default_rng API)
# - SciPy version: >=1.7.0 (for expit function)

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
import time
from scipy.special import expit

# ==========================================
# Core Simulation Functions (Vectorized Batch)
# ==========================================
# These functions are duplicated here to make the widget cell self-contained

def init_equilibrium_batch(N, M, rng):
    """Initialize random binary states for batch simulation."""
    return rng.integers(0, 2, size=(M, N), dtype=np.int8)

def update_step_batch(X, c_arr, beta, theta_arr, rng):
    """
    Perform one timestep update using Ising-like dynamics.
    Note: c_arr should have length N-1 (coupling between adjacent nodes).
    """
    M, N = X.shape
    left = np.zeros_like(X)
    right = np.zeros_like(X)
    left[:, 1:] = X[:, :-1] * c_arr
    right[:, :-1] = X[:, 1:] * c_arr
    neighbor_sum = left + right
    bias = beta * (neighbor_sum - theta_arr[None, :])
    p1 = expit(bias)
    U = rng.random(size=(M, N))
    X_new = (U < p1).astype(np.int8)
    return X_new

def compute_mutual_information_from_counts(counts, M):
    """Calculate mutual information I(A:X_i,t) from co-occurrence counts."""
    T, N, _, _ = counts.shape
    I = np.zeros((T, N), dtype=float)
    epsilon = 1e-12  # Avoid log(0)

    for t in range(T):
        for i in range(N):
            p_a_x = counts[t, i].astype(float) / M
            p_a = p_a_x.sum(axis=1)
            p_x = p_a_x.sum(axis=0)
            mi = 0.0
            for a in (0, 1):
                for x in (0, 1):
                    p = p_a_x[a, x]
                    if p <= epsilon:
                        continue
                    denom = p_a[a] * p_x[x]
                    if denom <= epsilon:
                        continue
                    mi += p * np.log2(p / denom)
            I[t, i] = mi
    return I

def persistence_times(I_arr, eps):
    """
    Compute persistence time (tau) for each node.
    Returns first timestep where I drops below threshold eps.
    """
    T, N = I_arr.shape
    tau = np.full(N, T, dtype=int)
    for i in range(N):
        below_indices = np.where(I_arr[:, i] < eps)[0]
        if below_indices.size > 0:
            tau[i] = below_indices[0]
        else:
            tau[i] = T  # Never decayed within T timesteps
    return tau

def run_simulation_chunked(N, c_arr, beta, theta_arr, M, T, source_j,
                           master_seed=12345, chunk_size=500, continuous_source=False):
    """
    Run batch simulation with memory-efficient chunking.
    Processes M simulations in chunks to avoid memory overflow.
    """
    main_rng = np.random.default_rng(master_seed)
    total_counts = np.zeros((T, N, 2, 2), dtype=np.int64)

    num_chunks = M // chunk_size + (1 if M % chunk_size != 0 else 0)

    for chunk_idx in range(num_chunks):
        current_chunk_size = min(chunk_size, M - chunk_idx * chunk_size)
        if current_chunk_size == 0:
            continue

        # Generate unique seed for reproducibility
        chunk_seed = main_rng.integers(0, 2**32 - 1)
        chunk_rng = np.random.default_rng(chunk_seed)

        X = init_equilibrium_batch(N, current_chunk_size, chunk_rng)
        A = chunk_rng.integers(0, 2, size=current_chunk_size, dtype=np.int8)
        X[:, source_j] = A

        for t in range(T):
            # Accumulate co-occurrence counts
            for a in (0, 1):
                mask_a = (A == a)
                sub = X[mask_a]
                if sub.size > 0:
                    total_counts[t, :, a, 1] += np.sum(sub, axis=0)
                    total_counts[t, :, a, 0] += sub.shape[0] - np.sum(sub, axis=0)

            X = update_step_batch(X, c_arr, beta, theta_arr, chunk_rng)

            if continuous_source:
                X[:, source_j] = A  # Continuous source clamping (if enabled)

    return total_counts

# ==========================================
# Interactive Visualization Function
# ==========================================

def interactive_two(pocket_strength=10, beta=1.5):
    """
    Interactive explorer for pocket strength and temperature (beta) parameters.

    Parameters:
        pocket_strength: Coupling strength in the pocket region (0-40)
        beta: Inverse temperature controlling noise/randomness (0.2-4.0)
    """
    # Simulation parameters
    N = 21              # Number of nodes
    T = 60              # Timesteps
    source_j = 10       # Source node (center)
    M = 1000            # Ensemble size (keep moderate for interactivity)
    chunk_size = 500    # Chunk size for memory efficiency
    theta = 1.0         # Node bias/threshold
    continuous_source = False  # Source drives once at t=0
    eps = 1e-3          # Threshold for tau calculation

    # Create coupling array with pocket in the center
    c_pocket = np.ones(N-1)  # Base coupling = 1.0
    mid = N // 2             # Center index
    half = 3                 # Pocket half-width
    # Apply strong coupling in pocket region (nodes mid-half to mid+half-1)
    c_pocket[mid-half:mid+half] = float(pocket_strength)

    # Run simulation
    t0 = time.perf_counter()
    counts = run_simulation_chunked(N, c_pocket, float(beta), np.ones(N)*theta,
                                    M, T, source_j, master_seed=20251123,
                                    chunk_size=chunk_size, continuous_source=continuous_source)
    I = compute_mutual_information_from_counts(counts, M)
    tau = persistence_times(I, eps=eps)
    elapsed = time.perf_counter() - t0

    # ==========================================
    # Visualization: 3-panel figure
    # ==========================================
    plt.figure(figsize=(10, 3))

    # Panel 1: Total information retention over time
    plt.subplot(1, 3, 1)
    S = I.sum(axis=1)  # Sum MI across all nodes
    plt.plot(S, '-o', markersize=3)
    plt.xlabel('Timestep')
    plt.ylabel('Sum I(A:i,t)')
    plt.title('Global Information Retention')
    plt.grid(True, alpha=0.3)

    # Panel 2: Heatmap of MI(t, node)
    plt.subplot(1, 3, 2)
    plt.imshow(I.T, origin='lower', aspect='auto', cmap='viridis', extent=[0, T, 0, N])
    plt.title(f'MI Heatmap (pocket={pocket_strength}, β={beta})')
    plt.xlabel('Timestep')
    plt.ylabel('Node index')
    plt.colorbar(shrink=0.6, label='I(A:X_i)')

    # Panel 3: Persistence times per node
    plt.subplot(1, 3, 3)
    plt.plot(tau, marker='o', linewidth=2, markersize=5)
    # Highlight pocket region with shaded area
    plt.axvspan(mid-half, mid+half-1, color='orange', alpha=0.2, label='Pocket')
    plt.xlabel('Node index')
    plt.ylabel('Persistence time (tau)')
    plt.title('Information Persistence per Node')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Print summary statistics
    inside_idx = list(range(mid-half, mid+half))  # Pocket nodes
    outside_idx = [i for i in range(N) if i not in inside_idx]  # Non-pocket nodes

    mean_inside = np.nanmean(tau[inside_idx])
    mean_outside = np.nanmean(tau[outside_idx])

    print(f"{'='*70}")
    print(f"Simulation Results:")
    print(f"  Pocket strength: {pocket_strength}")
    print(f"  Beta (inv. temp): {beta}")
    print(f"  Mean tau (inside pocket): {mean_inside:.2f} timesteps")
    print(f"  Mean tau (outside pocket): {mean_outside:.2f} timesteps")
    print(f"  Computation time: {elapsed:.2f}s")
    print(f"{'='*70}")

# ==========================================
# Widget Configuration and Display
# ==========================================
# Create sliders with descriptive parameters
p_slider = IntSlider(
    value=10,
    min=0,
    max=40,
    step=1,
    description='Pocket Strength',
    style={'description_width': '120px'}
)

b_slider = FloatSlider(
    value=1.5,
    min=0.2,
    max=4.0,
    step=0.1,
    description='Beta (inv. temp)',
    style={'description_width': '120px'}
)

# Initialize interactive widget
# Note: This will only work in Jupyter environments (Notebook, Lab, Colab)
print("Initializing interactive widget explorer...")
print("Adjust sliders to explore parameter space\n")
interact(interactive_two, pocket_strength=p_slider, beta=b_slider)


In [ ]:
# ==========================================
# INTERACTIVE WIDGET - COMPACT VERSION
# ==========================================
# This is a streamlined version of the interactive explorer.
# For the full version with detailed comments, see Cell 13.

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
import time

def interactive_two(pocket_strength=10, beta=1.5):
    """Interactive explorer for pocket strength and beta parameters."""
    N = 21
    T = 60
    source_j = 10
    M = 1000
    chunk_size = 500
    theta = 1.0
    continuous_source = False
    eps = 1e-3

    # Build pocket coupling
    c_pocket = np.ones(N-1)
    mid = N // 2
    half = 3
    c_pocket[mid-half:mid+half] = float(pocket_strength)

    # Run simulation
    t0 = time.perf_counter()
    counts = run_simulation_chunked(N, c_pocket, float(beta), np.ones(N)*theta,
                                    M, T, source_j, master_seed=20251123,
                                    chunk_size=chunk_size, continuous_source=continuous_source)
    I = compute_mutual_information_from_counts(counts, M)
    tau = persistence_times(I, eps=eps)
    elapsed = time.perf_counter() - t0

    # Visualize results
    S = I.sum(axis=1)
    plt.figure(figsize=(10, 3))

    plt.subplot(1, 3, 1)
    plt.plot(S, '-o', markersize=3)
    plt.xlabel('Timestep')
    plt.ylabel('Sum I(A:i,t)')
    plt.title('Global Information Retention')

    plt.subplot(1, 3, 2)
    plt.imshow(I.T, origin='lower', aspect='auto', cmap='viridis', extent=[0, T, 0, N])
    plt.title(f'I heatmap (p={pocket_strength}, β={beta})')
    plt.xlabel('Timestep')
    plt.ylabel('Node index')
    plt.colorbar(shrink=0.6)

    plt.subplot(1, 3, 3)
    plt.plot(tau, marker='o')
    plt.axvspan(mid-half, mid+half-1, color='orange', alpha=0.2)
    plt.xlabel('Node index')
    plt.ylabel('Persistence time')
    plt.title('tau per node')

    plt.tight_layout()
    plt.show()

    inside_idx = list(range(mid-half, mid+half))
    outside_idx = [i for i in range(N) if i not in inside_idx]
    mean_inside = np.nanmean(tau[inside_idx])
    mean_outside = np.nanmean(tau[outside_idx])
    print(f"pocket={pocket_strength} β={beta}  τ_in={mean_inside:.2f}  τ_out={mean_outside:.2f}  time={elapsed:.2f}s")

# Create sliders
p_slider = IntSlider(value=10, min=0, max=40, step=1, description='pocket')
b_slider = FloatSlider(value=1.5, min=0.2, max=4.0, step=0.1, description='beta')

# Initialize widget
interact(interactive_two, pocket_strength=p_slider, beta=b_slider)
